In [1]:
import pandas as pd
import sys 

sys.path.append('../src')

In [19]:
root_df = pd.read_csv('/Users/rt853/UoB-HICCS-2025/data/datasets/binary_datasets/brown/hobbies_binary.csv')
root_df.head()

,text,category,class
0,The 300th anniversaries of Staten Island ( thr...,reviews,0
1,"If necessary to replace both halves on grill ,...",hobbies,1
2,"`` How many is that , Jim '' ? ?",news,0
3,Most of the fire was directed by Cotten agains...,news,0
4,"`` You haven't got cancer '' , I said as stron...",romance,0


In [20]:
from baseline_model.model import init_model
from baseline_model.preprocess import vectorise_data
from sklearn.model_selection import train_test_split

In [21]:
model, model_type = init_model(4, 'binary', verbose=True)

In [22]:
train_df, test_df = train_test_split(root_df, test_size=0.2, random_state=42)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import ast

def label_2_idx(df, label_col):
    """
    Converts the labels in the label column to indices.
    """
    df[label_col] = df[label_col].astype('category')
    df[label_col] = df[label_col].cat.codes
    return df[label_col]

def vectorise_data(train_df, classification_type, vectorizer_type, 
        ngram_range, col_name):
    """
    Vectorises the data based on the vectorizer type.
    """
    ngram_range = ast.literal_eval(ngram_range)

    if vectorizer_type == 'tfidf':
        vectorizer = TfidfVectorizer(
            ngram_range=ngram_range, stop_words='english')
    elif vectorizer_type == 'count':
        vectorizer = CountVectorizer(
            ngram_range=ngram_range, stop_words='english')
    else:
        raise ValueError('Invalid vectorizer type')

    x = vectorizer.fit_transform(train_df[col_name])
    train_df[classification_type] = label_2_idx(train_df, classification_type)
    y = train_df[classification_type]
    return x, y, vectorizer

X, y , vectorizer = vectorise_data(train_df, 'class', 'tfidf', '(1,1)', 'text')

In [24]:
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [25]:
test_df['class'] = label_2_idx(test_df, 'class')
X_test = vectorizer.transform(test_df['text'])

In [26]:
preds = model.predict(X_test)

In [27]:
from sklearn.metrics import classification_report, accuracy_score

In [28]:
print(classification_report(test_df['class'], preds))

              precision    recall  f1-score   support

           0       0.93      1.00      0.97     10625
           1       0.84      0.12      0.22       843

    accuracy                           0.93     11468
   macro avg       0.89      0.56      0.59     11468
weighted avg       0.93      0.93      0.91     11468

